In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Importer la fonction de nettoyage
from csv_cleaner import clean_df

def load_and_clean_data(file_path, cleaning_threshold):
    """Charger les données et les nettoyer."""
    df = pd.read_csv(file_path)
    df_cleaned = clean_df(df, cleaning_threshold)
    return df_cleaned

# Charger et nettoyer les données
df_cleaned = load_and_clean_data('train.csv', 10)

# Séparer les colonnes prédictives (X) et la colonne cible (y)
X = df_cleaned.drop('Price', axis=1)
y = df_cleaned['Price']

# Séparer les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Créer des transformateurs pour les colonnes numériques et catégorielles
numeric_features = X_train.select_dtypes(include=[np.number]).columns
categorical_features = X_train.select_dtypes(exclude=[np.number]).columns

numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combinaison des transformateurs dans un ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Créer le pipeline complet avec prétraitement et modèle
full_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(n_estimators=100, random_state=42))
])

# Entraîner le pipeline avec le sous-ensemble d'entraînement
full_pipeline.fit(X_train, y_train)

# Prédictions sur l'ensemble de test
y_pred = full_pipeline.predict(X_test)

# Évaluation du modèle
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Afficher les résultats
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("R² Score:", r2)


Mean Squared Error (MSE): 13.798317389935498
Root Mean Squared Error (RMSE): 3.7146086455958587
R² Score: 0.8878727908131862


In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

# Importer la fonction de nettoyage des données
from csv_cleaner import clean_df

# Charger les données
df = pd.read_csv('train.csv')

# Nettoyer les données avec la fonction clean_df
cleaning_threshold = 10  # Ajustez ce seuil en fonction de vos besoins
df_cleaned = clean_df(df, cleaning_threshold)

# Séparer les colonnes prédictives (X) et la colonne cible (y)
X = df_cleaned.drop('Price', axis=1)
y = df_cleaned['Price']

# Séparer les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Définir les étapes de prétraitement pour les caractéristiques numériques et catégorielles
numerical_features = X_train.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X_train.select_dtypes(include=['object']).columns

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_transformer, numerical_features),
    ('cat', categorical_transformer, categorical_features)
])

# Créer le pipeline complet avec prétraitement et modèle
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(random_state=42))
])

# Définir la grille de paramètres pour la recherche d'hyperparamètres
param_grid = {
    'model__n_estimators': [50, 100],
    'model__max_depth': [None, 5, 10],
    'model__min_samples_split': [2, 5],
    'model__min_samples_leaf': [1]
}
# Configurer GridSearchCV avec le pipeline et la grille de paramètres
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='r2', n_jobs=-1, verbose=1)

# Entraîner le modèle en utilisant GridSearchCV
grid_search.fit(X_train, y_train)

# Afficher les meilleurs paramètres
print("Meilleurs hyperparamètres :", grid_search.best_params_)

# Prédictions sur l'ensemble de test avec le meilleur modèle trouvé
y_pred = grid_search.predict(X_test)

# Évaluation du modèle
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Afficher les résultats de l'évaluation
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("R² Score:", r2)


Fitting 5 folds for each of 12 candidates, totalling 60 fits
Meilleurs hyperparamètres : {'model__max_depth': None, 'model__min_samples_leaf': 1, 'model__min_samples_split': 2, 'model__n_estimators': 100}
Mean Squared Error (MSE): 13.798317389935498
Root Mean Squared Error (RMSE): 3.7146086455958587
R² Score: 0.8878727908131862
